## Calculates coverages for all forecasts

This code scales and forecasts for each day with data available. Then for each value of `t_0` and a forecast window of 1 year we calculate the `1-\alpha=0.1,...,0.99` confidence intervals. For each week in the 1 year forecast we report whether the `1-\alpha` confidence intervals captured the Fourier smoothed trap data point.

Start by importing necessary libraries and turning off warnings

In [2]:
import os, sys, importlib, glob, json
import pandas as pd

from scipy.stats import nbinom as nbinom
import itertools
sys.path.append( os.path.abspath(os.path.join('..')) )
import utils.forecast as forecast
import utils.utils as gen_utils

import warnings
warnings.filterwarnings('ignore')

# Opening config file
f = open('../fpaths_config.json')
paths = json.load(f)

raw_data_path = paths['raw_data']
smoothed_data_path = paths['smoothed_data']
coverages_path = paths["coverages_path"]


## Test combinations of scaler and forecast windows

For a prediction on day $t_0$, the scaling window contains the $[t_0-\text{scaling win}:t_0]$ days of trap data and nn predictions. The scaling window is used for several purposes:
* We estimate the probability of success in the negative binomial distribution (p) using the scaling window of trap data:
$$
p_{avg} = \frac{1}{n}\sum_{i=1}^n\frac{\mu_i}{\sigma_i^2}
$$
where $\mu_i, \sigma_i$ are the observed trap mean and standard deviation for week $i$ and $n=\lfloor \text{scaling win}/7 \rfloor$ is the number of weeks in the scaling window (since we have weekly, not daily, trap data)
* The scaling factor for the neural network predictions is calculated such that the average neural network predictions in the scaling window is equivalent to the average trap data in the same scaling window
$$
scaling_{rto} = \frac{\overline{\text{trap}}}{\overline{\text{nn}}}
$$
where $\overline{\text{trap}}$ is the average trap data in the scaling window and $\overline{\text{nn}}$ is the average neural network prediction in the scaling window.
* We use a scaling window of $90$ days.

Then for prediction day $t_0$ the forecast window contains the $[t_0:t_0+\text{forecast win}]$ days of trap data and nn predictions.
* The subset of neural network predictions $[t_0-\text{scaler win}:t_0+\text{forecast win}]$ is scaled by multiplying the $scaling_{rto}$ defined above
* Use the forecast window neural network predictions to estimate the variance $\sigma^2$ and number of successes $n$ in the negative binomial distribution.
$$
\sigma^2 = \frac{\hat{nn}}{p_{avg}};\quad\quad n = \frac{\hat{nn}^2}{\sigma^2-\hat{nn}}
$$
where $\hat{nn}$ are the neural network predictions (here representing the daily mean of the forecast window) and $p_{avg}$ is defined above.
* Convert the neural network predictions into a probablistic forecast using `nbinom.interval`
* For each week of trap data in the forecasting window, report wether the confidence interval captures the trap data point.
* We use a forecasting window of length $365$ days

Finally, we generate a csv file with columns Site, Scaler_win, Forecast_win, CIs that shows the coverage of each combination of site, scaling window, and forecast window.

In [11]:
importlib.reload(forecast)

if not os.path.exists(coverages_path):
    os.mkdir(coverages_path)

# Test airport weather compared to true weather and impact of Fourier smoothing
# Neural network predictions are always smoothed

weather_flags = ['site', 'airport']
smoothing_flags = [False, True]

weather_smoothing_flags = list(itertools.product(weather_flags, smoothing_flags))

for weather_smoothing_flag in weather_smoothing_flags:
    weather_flag, smoothing_flag = weather_smoothing_flag

    sites = ['Arboleda', 'Playa', 'La_Margarita', 'Villodas']

    scaler_wins = [13]
    forecast_wins = [52]

    scaler_forecast_wins = list(itertools.product(scaler_wins, forecast_wins))

    for site in sites:
        if smoothing_flag:
            nn_fil_name = '{}/{}_{}_smoothed_weekly_predictions.csv'.format(smoothed_data_path,site,weather_flag)
            ofil = '{}/{}_{}_smoothed_coverages.csv'.format(coverages_path, site, weather_flag)
        else:
            nn_fil_name = '{}/{}_{}_raw_weekly_predictions.csv'.format(raw_data_path,site,weather_flag)
            ofil = '{}/{}_{}_raw_coverages.csv'.format(coverages_path, site, weather_flag)

        print(site, weather_flag, smoothing_flag)
        site_info = [site, weather_flag, smoothing_flag]

        site_nn_data = gen_utils.load_csv(nn_fil_name)
        site_nn_data.Datetime = pd.to_datetime(site_nn_data.Datetime)
        
        if not os.path.exists(ofil):
            header = 'Site\tt_0\talpha\t'
            for i in range(1,53):
                header += 'wk{}\t'.format(i)
            header += '\n'
            with open(ofil, 'w') as f:
                f.write(header)
                f.close() 
        forecast.coverage_analysis(site_info, site_nn_data, scaler_forecast_wins, ofil)

Arboleda site False
Playa site False
La_Margarita site False
Villodas site False
Arboleda site True
Playa site True
La_Margarita site True
Villodas site True
Arboleda airport False
Playa airport False
La_Margarita airport False
Villodas airport False
Arboleda airport True
Playa airport True
La_Margarita airport True
Villodas airport True
